In [1]:
import sys, os, time
import shutil
%pdb
# %load_ext autoreload
# %autoreload 2
os.environ['FOUNDATION_RUN_MODE'] = 'jupyter'
os.environ['FOUNDATION_SAVE_DIR'] = '/is/ei/fleeb/workspace/chome/trained_nets'
os.environ['FOUNDATION_DATA_DIR'] = '/is/ei/fleeb/workspace/local_data'
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
import torchvision

from torch.utils.data import Dataset, DataLoader, TensorDataset
import gym
import numpy as np
# %matplotlib notebook
%matplotlib tk
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import imageio
import seaborn as sns
# plt.switch_backend('Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from bisect import bisect_left

# from IPython.display import HTML, Image

#from foundation.util import replicate, Cloner

from hybrid import get_model, get_data
from analyze import *

from tqdm import tqdm_notebook as tqdm

# plt.ioff()

np.set_printoptions(linewidth=120, suppress=True)

Automatic pdb calling has been turned ON


In [2]:
media_dir = '/is/ei/fleeb/workspace/media/hybrid/results'

In [39]:
M = Hybrid_Controller(media_dir).sort_by('date')

Found 95 runs


In [40]:
M.clear_filters()
pass

In [41]:
M.prep_info(name='model.pth.tar')

M.load_into('eval.pth.tar')

Selecting checkpoint: last
Loaded configs


In [51]:
def _get_stats(run):
    if 'eval' in run:
        run.stats = {
            'FID-prior': run.eval['FID-prior'],
            
            
#             'DCI-inf': run.eval['DCI']['informativeness_test'],
#             'DCI-cmp': run.eval['DCI']['completeness'],
        }
        
        if 'FID-hyb' in run.eval and run.eval['FID-hyb'] is not None:
            run.stats.update({
                'FID-hyb': run.eval['FID-hyb'],
                'FID-rec': run.eval['FID-rec'],
            })
        
        if 'IRS' in run.eval and run.eval['IRS'] is not None:
            run.stats.update({
                
                'IRS': run.eval['IRS']['IRS'],
                'DCI-dis': run.eval['DCI']['disentanglement'],
            })
M.map(_get_stats, safe=True)

# M.filter(lambda r: 'stats' in r and len(r.stats) and 'FID-prior' in r.stats)
M.filter(lambda r: 'stats' in r and len(r.stats) and 'IRS' in r.stats)
pass

M.store_selection('done')


In [52]:
# '3dshapes-fvae_0009-6027505-03_200129-125156'

In [53]:
M.show('all')

  0) 3dshapes-ae_0002-6026518-03_200129-001602_ckpt40
	Checkpoint: 40 (approx 120000 steps)
	 FID-prior : 51.53
	   FID-hyb : 34.46
	   FID-rec : 23.75
	       IRS : 0.56
	   DCI-dis : 0.28
	output.save_freq - 3000 (5000)
	training.val_freq - 3000 (5000)
	model.latent_reg_wt - 0 (100)

  1) 3dshapes-ae_0002-6026518-03_200129-001602_ckpt50
	Checkpoint: 50 (approx 150000 steps)
	 FID-prior : 51.24
	   FID-hyb : 33.95
	   FID-rec : 22.82
	       IRS : 0.55
	   DCI-dis : 0.28
	output.save_freq - 3000 (5000)
	training.val_freq - 3000 (5000)
	model.latent_reg_wt - 0 (100)

  2) 3dshapes-ae_0007-6027503-02_200129-124609_ckpt20
	Checkpoint: 20 (approx 60000 steps)
	 FID-prior : 130.76
	   FID-hyb : 63.80
	   FID-rec : 40.18
	       IRS : 0.58
	   DCI-dis : 0.29
	output.save_freq - 3000 (5000)
	training.step_limit - 60000 (150000)
	training.val_freq - 3000 (5000)
	model.latent_reg_wt - 0 (100)

  3) 3dshapes-dae_0012-6032710-00_200129-182006_ckpt1
	Checkpoint: 1 (approx 3000 steps)
	 FID-prior 

In [54]:
M.switch_selection('done')
len(M)

89

In [55]:
# M.invert()
# len(M)
# M.show()
# ' '.join(['_'.join(run.name.split('_')[:-1]) for run in M])

In [56]:
plt.close('all')
figaxs = {}

In [57]:
M.map(lambda r: r.meta.job[0], reduce=Counter)

Counter({'2': 18,
         '7': 13,
         '12': 5,
         '13': 5,
         '4': 16,
         '9': 13,
         '0': 14,
         '18': 3,
         '3': 2})

In [64]:
M.switch_selection('done')

# M.filter(lambda r: 'arch' in r.meta and r.meta.arch != 'double')

# M.filter_sel(slice(-10,None,None))
M.filter_min(ckpt=25)

M.filter_models('fwae', 'fvae', 'vae', 'wae')
# M.filter_models('dae', 'wae', 'vae', 'ae')
# M.filter_models('fdwae', 'fdvae', 'dvae', 'dwae', 'dae')
# M.filter_models('wgan')
# M.filter_jobs(94, 122)
# M.show('checkpoint', 'unique', )
# M.filter(lambda r: r.stats['DCI-dis'] > 0.75)

# M.sort_by(lambda runs: util.sort_by(runs, [run.stats['FID-rec'] for run in runs]))
# M.sort_by(lambda runs: util.sort_by(runs, [run.stats['FID-hyb'] for run in runs]))
# M.sort_by(lambda runs: util.sort_by(runs, [run.stats['DCI-dis'] for run in runs]))

# M.sort_by(lambda runs: util.sort_by(runs, [run.stats['IRS'] for run in runs]))
M.sort_by(lambda runs: util.sort_by(runs, [run.stats['DCI-dis'] for run in runs]))
pass

In [65]:
# plt.close('all')
# figaxs = {}
fig_stats = {
    'dis':['DCI-dis', 'IRS'], 
#     'fid2': ['FID-rec', 'FID-hyb'],
#     'fid': ['FID-prior', ],
    'fid': [ 'FID-rec', 'FID-hyb', 'FID-prior',],
#              'fid':['FID-rec', ],
            }
for i, (name,stats,) in enumerate(fig_stats.items()):
    if name not in figaxs:
        figaxs[name] = plt.subplots(figsize=(12,5))
    figaxs[name][1].cla()
    figaxs[name] = M.plot_stats(*stats, figax=figaxs.get(name,None))#, color='C{}'.format(i))
    plt.tight_layout()
# M.show('checkpoint', 'unique', manuals=[lambda r: 'Generator: {}'.format(r.config.model.generator._type)])
M.show('all', manuals=[lambda r: 'Generator: {}'.format(r.config.model.generator._type)])

  0) 3dshapes-wae_0002-6026518-02_200129-001602_ckpt50
	Checkpoint: 50 (approx 150000 steps)
	 FID-prior : 28.49
	   FID-hyb : 28.67
	   FID-rec : 23.99
	       IRS : 0.58
	   DCI-dis : 0.29
	output.save_freq - 3000 (5000)
	training.val_freq - 3000 (5000)
	model.latent_reg_wt - 10000 (100)
	Generator: double-dec

  1) 3dshapes-wae_0003-6026539-01_200129-002624_ckpt33
	Checkpoint: 33 (approx 99000 steps)
	 FID-prior : 38.61
	   FID-hyb : 38.76
	   FID-rec : 27.24
	       IRS : 0.58
	   DCI-dis : 0.31
	output.save_freq - 3000 (5000)
	training.val_freq - 3000 (5000)
	Generator: double-dec

  2) 3dshapes-wae_0002-6026518-02_200129-001602_ckpt32
	Checkpoint: 32 (approx 96000 steps)
	 FID-prior : 30.75
	   FID-hyb : 30.44
	   FID-rec : 26.25
	       IRS : 0.59
	   DCI-dis : 0.31
	output.save_freq - 3000 (5000)
	training.val_freq - 3000 (5000)
	model.latent_reg_wt - 10000 (100)
	Generator: double-dec

  3) 3dshapes-wae_0003-6026539-01_200129-002624_ckpt50
	Checkpoint: 50 (approx 150000 steps)

In [28]:
run = M[0]

In [15]:
# run.config.model.generator._type

In [16]:
# run.stats

In [17]:
# run.meta.arch